# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#!pip install python-dotenv

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

# Specify the exact path to your .env file in Google Drive
#env_path = "/content/drive/MyDrive/Ironhack/langchain_evaluation"

# Use find_dotenv() with the specified path
#_ = load_dotenv(find_dotenv(env_path))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [2]:
# Print statements to confirm API keys were loaded

print(f"OPENAI_API_KEY loaded: {bool(OPENAI_API_KEY)}")
print(f"LANGCHAIN_API_KEY loaded: {bool(LANGCHAIN_API_KEY)}")
print(f"HUGGINGFACEHUB_API_TOKEN loaded: {bool(HUGGINGFACEHUB_API_TOKEN)}")


OPENAI_API_KEY loaded: True
LANGCHAIN_API_KEY loaded: True
HUGGINGFACEHUB_API_TOKEN loaded: True


In [ ]:
# Print statements to confirm API keys were loaded
print(f"OPENAI_API_KEY: {OPENAI_API_KEY}")
print(f"LANGCHAIN_API_KEY: {LANGCHAIN_API_KEY}")
print(f"HUGGINGFACEHUB_API_TOKEN: {HUGGINGFACEHUB_API_TOKEN}")


In [4]:
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"]=LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"]="langchain_evaluation-IronHack_Lab"
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

### Example 1

#### Create our QandA application

In [8]:
#!pip install langchain_openai

In [9]:
#!pip install langchain-community

In [10]:
#!pip install langchain-huggingface

In [11]:
#!pip install -U langchain-openai

In [6]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI


In [7]:
file = '/content/drive/MyDrive/Ironhack/langchain_evaluation/OutdoorClothingCatalog_1000.csv' #'../data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [14]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [15]:
#!pip install docarray

In [19]:
#!pip install --upgrade langchain pydantic

In [8]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})
).from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [9]:
llm = ChatOpenAI(temperature = 0.0, api_key=OPENAI_API_KEY)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [10]:
data[10]

Document(metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\r\n\r\nSize & Fit\r\n- Pants are Favorite Fit: Sits lower on the waist.\r\n- Relaxed Fit: Our most generous fit sits farthest from the body.\r\n\r\nFabric & Care\r\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\r\n\r\nAdditional Features\r\n- Relaxed fit top with raglan sleeves and rounded hem.\r\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\r\n\r\nImported.")

In [11]:
data[11]

Document(metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [12]:
from langchain.prompts import PromptTemplate

In [13]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


<ipython-input-13-6c6267e67e31>:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
<ipython-input-13-6c6267e67e31>:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='Yes, it is available in 3 different colors: gray, navy, and black.'


In [25]:
#!pip install --upgrade langchain

#### LLM-Generated examples

In [14]:
from langchain.evaluation.qa import QAGenerateChain

In [15]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [16]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [17]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [18]:
new_examples[0]

{'qa_pairs': {'query': "What is the recommended sizing advice for the Women's Campside Oxfords based on the information provided in the document?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. For half sizes that are not offered, it is recommended to order up to the next whole size."}}

In [19]:
data[0]

Document(metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \r\n\r\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \r\n\r\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \r\n\r\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \r\n\r\nQuestions? Please contact us for any inquiries.")

In [20]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the recommended sizing advice for the Women's Campside Oxfords based on the information provided in the document?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. For half sizes that are not offered, it is recommended to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are the key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece, as described in the document?",
  'answer': 'The key features of the swimsuit include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom for secure fit and coverage, and the recommendation

#### Combine examples

In [26]:
# examples += new_example
examples += d_flattened

In [27]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [23]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Based on the provided context, the Cozy Comfort Pullover Set does not have side pockets. It has side seam pockets, a back zip pocket, two elastic mesh water bottle pockets, and a top compartment with a pocket with a double-seal zipper for quick access.'}

### Manual Evaluation - Fun part

In [28]:
import langchain
langchain.debug = True

In [29]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "Side seam pockets and back zip pocket, with mesh insert for quick drainage.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-seal zipper for quick access.\r\nSide<<<<>>>>>All pockets have sturdy pocket bags and offer plenty of room for a wallet, cell phone and more.\r\n\r\nGusseted crotch for ease of movement.\r\n\r\nImported.<<<<>>>>>Two elastic mesh water bottle pockets.\r\nTop compartment includes pocket with double-se"
}
[llm/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'}

In [30]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [31]:
examples += d_flattened

In [32]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the recommended sizing advice for the Women's Campside Oxfords based on the information provided in the document?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. For half sizes that are not offered, it is recommended to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are the key features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece, as described in the document?",
  'answer': 'The key features of the swimsuit include bright colo

In [33]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...

> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [34]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What is the recommended sizing advice for the Women's Campside Oxfords based on the information provided in the document?",
  'answer': "The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. For half sizes that are not offered, it is recommended to order up to the next whole size.",
  'result': "I'm sorry, but the Women's Campside Oxfords are not mentioned in the provided information. I don't have specific sizing advice for that product based on the details given."},
 {'que

In [35]:
from langchain.evaluation.qa import QAEvalChain

In [36]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [37]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [38]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [39]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set has side seam pockets and a back zip pocket, as well as two elastic mesh water bottle pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the recommended sizing advice for the Women's Campside Oxfords based on the information provided in the document?
Real Answer: The recommended sizing advice for the Women's Campside Oxfords is to order your regular shoe size. For half sizes that are not offered, it is recommended to order up to the next whole size.
Predicted Answer: I'm sorry, but the Women's Campside Oxfords are not mentioned in the provided information. I don't have specific sizing advice for that product based on th

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader("/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt")
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0.0, api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/usr/local/lib/python3.11/dist-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [41]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally founded as a Dutch trading post called New Amsterdam in 1626. In 1664, the city came under British control and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [42]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally founded as a Dutch trading post called New Amsterdam in 1626. In 1664, the city came under British control and was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='d06b7a7f-fa79-42f6-bf31-4e9a83b97d01', metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [43]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [44]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [45]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [46]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [47]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': [Document(id='45788da4-f571-460f-bee1-3935eac2e07f', metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and

In [ ]:
#!pip install --no-cache-dir recordclass

In [ ]:
#!pip install ragas==0.1.9

In [51]:
from pydantic import BaseModel

In [60]:
from ragas.integrations.langchain import EvaluatorChain
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [61]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'source_documents': [Document(id='45788da4-f571-460f-bee1-3935eac2e07f', metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston

**Faithfulness**

In [131]:
# Map keys as defined
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        if old_key == "source_documents":  # Handle contexts specifically
            # Extract 'page_content' from each Document and ensure all are strings
            list_context = [doc.page_content for doc in result[old_key]]
            result_updated[new_key] = ' '.join(list_context)

        else:
            result_updated[new_key] = result[old_key]

In [96]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

In [88]:
import nest_asyncio
nest_asyncio.apply()

In [98]:
eval_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

In [97]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness"]

0.5

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [114]:
# Map keys as defined
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

fake_result = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        if old_key == "source_documents":  # Handle contexts specifically
            # Extract 'page_content' from each Document and ensure all are strings
            list_context = [doc.page_content for doc in result[old_key]]
            fake_result[new_key] = ' '.join(list_context)

        else:
            fake_result[new_key] = result[old_key]

In [115]:
fake_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

In [116]:
fake_result["answer"] = "we are the champions"

In [117]:
fake_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'we are the champions',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New York City residents in 2020 (5,141,538, or 5

In [118]:
#fake_result = result.copy()
#fake_result["answer"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness"]

nan

In [122]:
fake_result_2 = fake_result.copy()
fake_result_2["answer"] = "Brooklyn is the biggest borough of NY"
eval_result = faithfulness_chain(fake_result_2)
eval_result["faithfulness"]

0.0

In [123]:
fake_result_2

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Brooklyn is the biggest borough of NY',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New York City residents in 2020

**Context Relevancy**

In [132]:
# Rename 'answer' key to 'ground_truth'
result_updated_gt={}
result_updated_gt=result_updated.copy() # Create a copy to avoid modifying the original dictionary
result_updated_gt["ground_truth"] = result_updated.pop("answer") # Assign the value of the key 'answer' to new key named 'ground_truth'

# Verify the updated result structure
print(result_updated_gt)

{'question': 'Which borough of New York City has the highest population?', 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolit

In [136]:
eval_result = context_recall_chain(result_updated_gt)
eval_result["context_recall"]

1.0

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [137]:
fake_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'we are the champions',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New York City residents in 2020 (5,141,538, or 5

In [138]:
# Create a new fake_result_03 based on the current fake_result structure
fake_result_03 = fake_result.copy()  # Start with a copy of fake_result

# Update the answer to the correct response
fake_result_03["answer"] = "Manhattan (New York County) has the highest population density of any borough in New York City."

# Add the ground_truth key with the correct response
fake_result_03["ground_truth"] = "Manhattan (New York County) has the highest population density of any borough in New York City."

# Modify contexts to be a single irrelevant string
fake_result_03["contexts"] = "I love christmas"

# Print the updated fake_result_03 for verification
print(fake_result_03)

{'question': 'Which borough of New York City has the highest population?', 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.', 'contexts': 'I love christmas', 'ground_truth': 'Manhattan (New York County) has the highest population density of any borough in New York City.'}


In [139]:
#from langchain.schema import Document
#fake_result = result.copy()
#fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result_03)
eval_result["context_recall"]

0.0

In [142]:
fake_result_04 = fake_result_03.copy()
fake_result_04["contexts"] = (
    "New York City has a large population, and Manhattan is one of its boroughs. "
    "However, Brooklyn also has a significant population density."
)

In [143]:
fake_result_04

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': 'New York City has a large population, and Manhattan is one of its boroughs. However, Brooklyn also has a significant population density.',
 'ground_truth': 'Manhattan (New York County) has the highest population density of any borough in New York City.'}

In [144]:
eval_result = context_recall_chain(fake_result_04)
eval_result["context_recall"]

0.0

In [145]:
fake_result_05 = fake_result_04.copy()
fake_result_05["contexts"] = (
    "Manhattan has a high population density. "
    "Brooklyn also has a significant population density, but Manhattan's is higher."
)

In [146]:
fake_result_05

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "Manhattan has a high population density. Brooklyn also has a significant population density, but Manhattan's is higher.",
 'ground_truth': 'Manhattan (New York County) has the highest population density of any borough in New York City.'}

In [147]:
eval_result = context_recall_chain(fake_result_05)
eval_result["context_recall"]

1.0

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [149]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

In [150]:
predictions

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190'],
  'result': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'source_documents': [Document(id='45788da4-f571-460f-bee1-3935eac2e07f', metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houst

In [153]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

In [158]:
predictions

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190'],
  'result': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'source_documents': [Document(id='45788da4-f571-460f-bee1-3935eac2e07f', metadata={'source': '/content/drive/MyDrive/Ironhack/langchain_evaluation/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houst

In [156]:
# Map keys as defined
key_mapping_p = {
    "query": "question",
    "ground_truths": "ground_truth",
    "result": "answer",
    "source_documents": "contexts"
}

formatted_predictions = {}
for old_key, new_key in key_mapping_p.items():
    if old_key in predictions:
        if old_key == "source_documents":  # Handle contexts specifically
            # Extract 'page_content' from each Document and ensure all are strings
            list_context = [doc.page_content for doc in predictions[old_key]]
            formatted_predictions[new_key] = ' '.join(list_context)

        else:
            formatted_predictions[new_key] = predictions[old_key]

In [159]:
# Define the key mapping
key_mapping_p = {
    "query": "question",
    "ground_truths": "ground_truth",
    "result": "answer",
    "source_documents": "contexts",
}

# Initialize the list to store formatted predictions
formatted_predictions = []

# Iterate over each prediction in the list
for prediction in predictions:
    formatted_prediction = {}
    for old_key, new_key in key_mapping_p.items():
        if old_key == "source_documents":
            # Handle 'source_documents' specifically
            list_context = [doc.page_content for doc in prediction.get(old_key, [])]
            formatted_prediction[new_key] = " ".join(list_context)
        else:
            # Map other keys directly
            formatted_prediction[new_key] = prediction.get(old_key)
    formatted_predictions.append(formatted_prediction)

# Print the formatted predictions
formatted_predictions

[{'question': 'What is the population of New York City as of 2020?',
  'ground_truth': ['8,804,190'],
  'answer': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropo

In [161]:
# evaluate
print("evaluating...")
#r = faithfulness_chain.evaluate(examples, predictions)
r = await faithfulness_chain.abatch(formatted_predictions)
r

evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'ground_truth': ['8,804,190'],
  'answer': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropo

In [162]:
# evaluate context recall
print("evaluating...")
cr = await context_recall_chain.abatch(formatted_predictions)
cr

evaluating...


[{'question': 'What is the population of New York City as of 2020?',
  'ground_truth': ['8,804,190'],
  'answer': 'The population of New York City as of 2020 was 8,804,190 residents.',
  'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropo